In [2]:
import pandas as pd

In [8]:
master_orders_df=pd.read_csv(r"C:\Users\asus\Desktop\OZI project\master_orders_dataset.csv")
#jan orders
jan_orders = master_orders_df[
    (master_orders_df['order_date'] >= '2024-01-01') &
    (master_orders_df['order_date'] <= '2024-01-31')
].copy()


In [12]:
#good experience flag
jan_orders['is_good_experience'] = (
    (jan_orders['rating'] >= 4) &
    (jan_orders['actual_delivery_mins'] <= jan_orders['promised_time_mins'])
)
jan_orders['is_good_experience'].value_counts(dropna=False)



is_good_experience
False    17788
True     12881
Name: count, dtype: int64

In [ ]:
#high value (High Value User: A customer whose total monetary spend in January exceeded the median January spend across all users.)
jan_user_spend = (
    jan_orders
    .groupby('user_id')['order_amount']
    .sum()
    .reset_index(name='total_jan_spend')
)
#median jan spend
median_jan_spend = jan_user_spend['total_jan_spend'].median()
median_jan_spend


np.float64(4720.0)

In [17]:
#Flag High Value Users
jan_user_spend['is_high_value'] = (
    jan_user_spend['total_jan_spend'] > median_jan_spend
)


#Merge High Value Flag Back to January Orders
jan_orders = jan_orders.merge(
    jan_user_spend[['user_id', 'is_high_value']],
    on='user_id',
    how='left'
)


In [22]:
jan_orders['is_high_value'].value_counts()


is_high_value
True     22160
False     8509
Name: count, dtype: int64

In [25]:
# Recreate Happy + High Value users (January)
happy_high_value_users_jan = jan_orders[
    (jan_orders['is_high_value'] == True) &
    (jan_orders['is_good_experience'] == True)
].copy()

happy_high_value_users_jan.to_csv(
    "happy_high_value_users_january.csv",
    index=False
)

print("✅ happy_high_value_users_january.csv saved successfully")


✅ happy_high_value_users_january.csv saved successfully
